In [54]:
import polars as pl
from pathlib import Path

In [55]:
files_dir = Path("../files")  # Dossier au même niveau que le répertoire actuel

In [56]:
# Charger le CSV avec les bonnes colonnes
df = pl.read_csv(
    files_dir/ "nom_prenoms_lower_trim.csv",
    separator=",",  # Ajuster selon le séparateur réel
    new_columns=["nom", "prenoms", "sexe", "age"],
    dtypes={
        "nom": pl.Utf8,
        "prenoms": pl.Utf8,
        "sexe": pl.Categorical,  # Optimisation mémoire
        "age": pl.UInt8
    }
)

C:\Users\USER\AppData\Local\Temp\ipykernel_1660\1450877043.py:2: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  df = pl.read_csv(


In [57]:
df

nom,prenoms,sexe,age
str,str,cat,u8
"""lago""","""serge""","""Masculin""",43
"""lago""","""renaud""","""Masculin""",40
"""lago""","""therese""","""Féminin""",68
"""bode""","""charleine""","""Féminin""",25
"""dina""","""dina""","""Féminin""",27
…,…,…,…
"""diallo""","""maiga""","""Masculin""",46
"""diallo""","""amina""","""Féminin""",28
"""diallo""","""ramata""","""Féminin""",21


In [59]:
df_exploded = df.with_columns(
    pl.col("prenoms").str.split(" ").alias("prenoms_split")
).explode("prenoms_split").rename({"prenoms_split": "prenom_explode"})

In [60]:
df_exploded

nom,prenoms,sexe,age,prenom_explode
str,str,cat,u8,str
"""lago""","""serge""","""Masculin""",43,"""serge"""
"""lago""","""renaud""","""Masculin""",40,"""renaud"""
"""lago""","""therese""","""Féminin""",68,"""therese"""
"""bode""","""charleine""","""Féminin""",25,"""charleine"""
"""dina""","""dina""","""Féminin""",27,"""dina"""
…,…,…,…,…
"""diallo""","""maiga""","""Masculin""",46,"""maiga"""
"""diallo""","""amina""","""Féminin""",28,"""amina"""
"""diallo""","""ramata""","""Féminin""",21,"""ramata"""


In [61]:
stats = (
    pl.concat([
        df_exploded.select(
            pl.col("prenom_explode").alias("nom"), 
            "sexe"
        ).with_columns(type=pl.lit("prenoms")),
        
        df.select(
            pl.col("nom").alias("nom"),
            "sexe"
        ).with_columns(type=pl.lit("famille"))
    ])
    .group_by("nom", "sexe", "type")
    .agg(count=pl.len())
    .pivot(
        values="count",
        index="nom",
        columns=["sexe", "type"],
        aggregate_function="sum"
    )
    .fill_null(0)
    .rename({
        '{"Féminin","famille"}': "nombre_femme_nom_famille",
        '{"Féminin","prenoms"}': "nombre_femme_prenom",
        '{"Masculin","famille"}': "nombre_homme_nom_famille",
        '{"Masculin","prenoms"}': "nombre_homme_prenom"  # Correction orthographique
    })
    
)


C:\Users\USER\AppData\Local\Temp\ipykernel_1660\2881166339.py:15: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


In [62]:
stats

nom,nombre_femme_prenom,nombre_homme_nom_famille,nombre_homme_prenom,nombre_femme_nom_famille
str,u32,u32,u32,u32
"""djenienba""",5,0,0,1
"""saindrin""",0,3,0,2
"""otiniel""",50,2,873,1
"""binjalerne""",1,0,0,0
"""vasikiri""",0,1,4,0
…,…,…,…,…
"""deasoi""",1,0,0,0
"""falafolo""",0,0,1,0
"""yode sarah""",0,0,0,1


In [63]:
prenom_unique_stats = (
    df_exploded.filter(pl.col("prenoms") == pl.col("prenom_explode"))
    .group_by("prenom_explode")
    .agg(prenom_unique=pl.len())
)

In [64]:
prenom_unique_stats

prenom_explode,prenom_unique
str,u32
"""wodjowontchai""",1
"""gdrh""",1
"""puire""",2
"""abdoulachirou""",1
"""ramirie""",1
…,…
"""goulirant""",1
"""direninan""",1
"""chafobi""",1


In [65]:
final_stats = (
    stats.join(
        prenom_unique_stats,
        left_on="nom",
        right_on="prenom_explode",
        how="left"
    )
    .with_columns(
        partie_prenom=pl.col("nombre_homme_prenom") + pl.col("nombre_femme_prenom"),
        nom_famille=pl.col("nombre_homme_nom_famille") + pl.col("nombre_femme_nom_famille"),
        prenom_unique=pl.coalesce("prenom_unique", 0)
    )
    .select([
        "nom",
        "partie_prenom",
        "nom_famille",
        "prenom_unique",
        "nombre_homme_prenom",
        "nombre_homme_nom_famille",
        "nombre_femme_prenom",
        "nombre_femme_nom_famille"
    ])
)

In [66]:
final_stats

nom,partie_prenom,nom_famille,prenom_unique,nombre_homme_prenom,nombre_homme_nom_famille,nombre_femme_prenom,nombre_femme_nom_famille
str,u32,u32,u32,u32,u32,u32,u32
"""djenienba""",5,1,4,0,0,5,1
"""saindrin""",0,5,0,0,3,0,2
"""otiniel""",923,3,218,873,2,50,1
"""binjalerne""",1,0,0,0,0,1,0
"""vasikiri""",4,1,4,4,1,0,0
…,…,…,…,…,…,…,…
"""deasoi""",1,0,0,0,0,1,0
"""falafolo""",1,0,1,1,0,0,0
"""yode sarah""",0,1,0,0,0,0,1


In [67]:
final_stats.write_parquet(
     files_dir / "nom_prenoms_overlay_vf_lower_trim.parquet",
       compression="zstd"
)